# A notebook for exploring the data and merging in other data sets

In [1]:
import pandas as pd

In [2]:
clean_crime_data_path = r'clean_data/stl_crime_data.txt'
neighborhood_demographics = r'additional_data/neighborhood_demographics_2010.csv'

In [9]:
crime_df = pd.read_csv(clean_crime_data_path, sep='\t', index_col=0)
neighborhood_df = pd.read_csv(neighborhood_demographics)

In [35]:
crime_neighborhoods = list(set(crime_df.NeighborhoodName))

In [36]:
demo_neighborhoods = list(set(neighborhood_df.Neighborhood))

In [37]:
len(crime_neighborhoods)

89

In [38]:
len(demo_neighborhoods)

79

### Create a standardized neighborhood mapping

In [56]:
# create initial mapping from the intersection between the two
# neighborhood_mapping = {x:x for x in neighborhood_intersection}
neighborhood_mapping = {}

# now fix some others to make them standard
neighborhood_mapping['Botanical Garden'] = 'Botanical Heights'
neighborhood_mapping['Cardonlet'] = 'Carondelet'
neighborhood_mapping['Carondelet Park'] = 'Carondelet'
neighborhood_mapping['Grand Center'] = 'Covenant Blu - Grand Center'
neighborhood_mapping['Fairground Neighborhood'] = 'Fairground'
neighborhood_mapping['Fairgrounds Park'] = 'Fairground'
neighborhood_mapping['Forest Park'] = 'Forest Park Southeast'
neighborhood_mapping['Forest Park SE'] = 'Forest Park Southeast'
neighborhood_mapping['Hi-Point'] = 'Hi-Pointe'
neighborhood_mapping['Jeff Vanderlou'] = 'JeffVanderlou'
neighborhood_mapping['Lasalle'] = 'Lasalle Park'
neighborhood_mapping['Mark Twain I-70 Ind.'] = 'Mark Twain/I-70 Industrial'
neighborhood_mapping['Near N. Riverfront'] = 'Near North Riverfront'
neighborhood_mapping['O’Fallon'] = "O'Fallon"
neighborhood_mapping["O'Fallon Park"] = "O'Fallon"
neighborhood_mapping['Peabody-Darst-Webbe'] = 'Peabody Darst Webbe'
neighborhood_mapping['Penrose Park'] = 'Penrose'
neighborhood_mapping['The Great Ville'] = 'Greater Ville'
neighborhood_mapping['Vandenventer'] = 'Vandeventer'
neighborhood_mapping['Wells-Goodfellow'] = 'Wells/Goodfellow'
neighborhood_mapping['West Ellendale'] = 'Ellendale'

neighborhood_mapping['Wydown-Skinker'] = 'Wydown Skinker'
neighborhood_mapping['Wydown/Skinker'] = 'Wydown Skinker'

# removing parks which aren't in a certain neighborhood
neighborhood_mapping['Calvary-Bellefontaine Cemetaries'] = 'Park'
neighborhood_mapping['Tower Grove Park'] = 'Park'
neighborhood_mapping['Wilmore Park'] = 'Park'

In [57]:
standard_neighborhoods = []

for neighborhood in crime_neighborhoods:
    if neighborhood in neighborhood_mapping:
        print(f"Swapped {neighborhood} for {neighborhood_mapping[neighborhood]}")
        standard_neighborhoods.append(neighborhood_mapping[neighborhood])
    else:
        print(f"No swapping needed for {neighborhood}")
        standard_neighborhoods.append(neighborhood)

No swapping needed for Vandeventer
No swapping needed for Franz Park
No swapping needed for The Ville
No swapping needed for Baden
No swapping needed for Midtown
Swapped West Ellendale for Ellendale
No swapping needed for Downtown
No swapping needed for Lewis Place
No swapping needed for Lafayette Square
No swapping needed for Carr Square
Swapped Wilmore Park for Park
Swapped Tower Grove Park for Park
No swapping needed for Walnut Park West
No swapping needed for Benton Park West
Swapped Lasalle for Lasalle Park
No swapping needed for Princeton Heights
No swapping needed for Lindenwood Park
No swapping needed for Compton Heights
Swapped Fairground Neighborhood for Fairground
No swapping needed for Bevo Mill
No swapping needed for Visitation Park
No swapping needed for Clifton Heights
No swapping needed for Benton Park
Swapped Mark Twain I-70 Ind. for Mark Twain/I-70 Industrial
No swapping needed for South Hampton
No swapping needed for Tower Grove East
No swapping needed for Kings Oak


In [58]:
standard_neighborhoods.sort()

The number of neighborhoods in St. Louis is 79, plus the "Unknown" and "Park" categories gets us 81, so our mapping is probably alright for most purposes

In [62]:
len(set(standard_neighborhoods))

81

Now to swap out the neighborhoods in the dataframes for their standardized counterparts

In [73]:
def standardize_neighborhood(neighborhood, neighborhood_mapping=neighborhood_mapping):
    try:
        return neighborhood_mapping['neighborhood']
    except KeyError:
        return neighborhood

crime_df['StandardNeighborhoodName'] = crime_df.NeighborhoodName.map(standardize_neighborhood)
neighborhood_df['StandardNeighborhoodName'] = neighborhood_df.Neighborhood.map(standardize_neighborhood)

### Reduce the crime dataset down to the useful bits, summarize crime by Neighborhood and Type of Crime

In [80]:
crime_df.columns

Index(['FileName', 'CADAddress', 'CADStreet', 'CodedMonth', 'Complaint',
       'Count', 'Crime', 'ShortCrimeCode', 'UCRType', 'UCRCrime',
       'DateOccured', 'Description', 'District', 'FlagAdministrative',
       'FlagCleanup', 'FlagCrime', 'FlagUnfounded', 'ILEADSAddress',
       'ILEADSStreet', 'LocationComment', 'LocationName', 'Neighborhood',
       'NeighborhoodName', 'NeighborhoodPrimaryDistrict',
       'NeighborhoodAddlDistrict', 'Latitude', 'Longitude',
       'StandardNeighborhoodName'],
      dtype='object')

In [119]:
columns_to_keep = ['DateOccured', 'Count', 'UCRCrime', 'StandardNeighborhoodName', 
                   'Latitude', 'Longitude', 'FlagCrime', 'FlagUnfounded']

df1 = crime_df[columns_to_keep]
df2 = neighborhood_df

### Merge the population data into the crime data

In [120]:
df = pd.merge(df1, df2, how='inner', on='StandardNeighborhoodName')

In [122]:
df.head(10)

,DateOccured,Count,UCRCrime,StandardNeighborhoodName,Latitude,Longitude,FlagCrime,FlagUnfounded,Neighborhood,Population,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor
0,01/01/2000 20:58,1,Sex Offenses,Fountain Park,38.658845,-90.260116,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
1,03/23/2008 17:00,1,Motor vehicle theft,Fountain Park,38.651938,-90.260857,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
2,04/01/2008 00:01,1,Fraud,Fountain Park,38.659750,-90.260558,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
3,04/01/2008 17:20,1,Larceny-theft,Fountain Park,38.655478,-90.257060,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
4,04/01/2008 18:00,1,Larceny-theft,Fountain Park,38.655254,-90.262784,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
5,04/01/2008 21:06,1,Liquor Laws,Fountain Park,38.655254,-90.262784,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
6,04/02/2008 17:00,1,Larceny-theft,Fountain Park,38.659898,-90.261975,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
7,04/03/2008 17:18,1,Other,Fountain Park,38.653877,-90.262053,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
8,04/04/2008 13:00,1,Motor vehicle theft,Fountain Park,38.659868,-90.256499,N,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
9,04/04/2008 13:00,-1,Motor vehicle theft,Fountain Park,38.659868,-90.256499,N,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North


### Clean up the set a bit to get one valid crime per row

Get rid of any rows that have a negative count, any where the FlagCrime is Y (only keep new crimes), and when FlagUnfounded is N (making sure the crime isn't unfounded).  I'm thinking of these like my rough set of "valid" crimes to be performing ML on

In [170]:
one_counts = df.Count == 1
y_flag_crime = df.FlagCrime == 'Y'
n_flag_unfounded = df.FlagUnfounded == 'N'

no_negative_counts = df[one_counts & y_flag_crime & n_flag_unfounded]

In [171]:
no_negative_counts.head(10)

,DateOccured,Count,UCRCrime,StandardNeighborhoodName,Latitude,Longitude,FlagCrime,FlagUnfounded,Neighborhood,Population,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor
0,01/01/2000 20:58,1,Sex Offenses,Fountain Park,38.658845,-90.260116,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
1,03/23/2008 17:00,1,Motor vehicle theft,Fountain Park,38.651938,-90.260857,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
2,04/01/2008 00:01,1,Fraud,Fountain Park,38.659750,-90.260558,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
3,04/01/2008 17:20,1,Larceny-theft,Fountain Park,38.655478,-90.257060,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
4,04/01/2008 18:00,1,Larceny-theft,Fountain Park,38.655254,-90.262784,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
5,04/01/2008 21:06,1,Liquor Laws,Fountain Park,38.655254,-90.262784,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
6,04/02/2008 17:00,1,Larceny-theft,Fountain Park,38.659898,-90.261975,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
7,04/03/2008 17:18,1,Other,Fountain Park,38.653877,-90.262053,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
10,04/04/2008 13:00,1,Motor vehicle theft,Fountain Park,38.659868,-90.256499,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North
11,04/05/2008 11:15,1,Aggravated Assault,Fountain Park,38.660401,-90.260948,Y,N,Fountain Park,1484,1.1,97.4,0.5,0.0,0.0,1.2,North


Now remove any duplicate columns

In [214]:
deduping_columns = ['DateOccured', 'UCRCrime', 'StandardNeighborhoodName',
                    'Latitude', 'Longitude', 'White', 'Black', 
                    'Hispanic/Latino2', 'AIAN1', 'Asian', 'Mixed Race', 'Corridor']

In [215]:
to_dedup_df = no_negative_counts[deduping_columns]

In [216]:
mask = to_dedup_df.duplicated(keep=False)

In [217]:
unique_crimes_df = to_dedup_df[~mask]

In [218]:
unique_crimes_df.shape

(336615, 12)

In [219]:
unique_crimes_df.head(10)

,DateOccured,UCRCrime,StandardNeighborhoodName,Latitude,Longitude,White,Black,Hispanic/Latino2,AIAN1,Asian,Mixed Race,Corridor
0,01/01/2000 20:58,Sex Offenses,Fountain Park,38.658845,-90.260116,1.1,97.4,0.5,0.0,0.0,1.2,North
1,03/23/2008 17:00,Motor vehicle theft,Fountain Park,38.651938,-90.260857,1.1,97.4,0.5,0.0,0.0,1.2,North
2,04/01/2008 00:01,Fraud,Fountain Park,38.659750,-90.260558,1.1,97.4,0.5,0.0,0.0,1.2,North
3,04/01/2008 17:20,Larceny-theft,Fountain Park,38.655478,-90.257060,1.1,97.4,0.5,0.0,0.0,1.2,North
4,04/01/2008 18:00,Larceny-theft,Fountain Park,38.655254,-90.262784,1.1,97.4,0.5,0.0,0.0,1.2,North
5,04/01/2008 21:06,Liquor Laws,Fountain Park,38.655254,-90.262784,1.1,97.4,0.5,0.0,0.0,1.2,North
6,04/02/2008 17:00,Larceny-theft,Fountain Park,38.659898,-90.261975,1.1,97.4,0.5,0.0,0.0,1.2,North
7,04/03/2008 17:18,Other,Fountain Park,38.653877,-90.262053,1.1,97.4,0.5,0.0,0.0,1.2,North
10,04/04/2008 13:00,Motor vehicle theft,Fountain Park,38.659868,-90.256499,1.1,97.4,0.5,0.0,0.0,1.2,North
11,04/05/2008 11:15,Aggravated Assault,Fountain Park,38.660401,-90.260948,1.1,97.4,0.5,0.0,0.0,1.2,North


In [220]:
unique_crimes_df.to_csv('clean_data/unique_crimes.csv')

### Now create some features to train on

Such as...

- year
- month
- day
- time
- ??

In [221]:
from dateutil.parser import parse

In [222]:
unique_crimes_df['DateOccured'] = unique_crimes_df['DateOccured'].map(lambda x: parse(x))

/Users/k784648/miniconda3/envs/dataingestion/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [223]:
unique_crimes_df['Year'] = unique_crimes_df['DateOccured'].map(lambda x: x.year)
unique_crimes_df['Month'] = unique_crimes_df['DateOccured'].map(lambda x: x.month)
unique_crimes_df['Day'] = unique_crimes_df['DateOccured'].map(lambda x: x.day)
unique_crimes_df['Hour'] = unique_crimes_df['DateOccured'].map(lambda x: x.hour)

/Users/k784648/miniconda3/envs/dataingestion/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/k784648/miniconda3/envs/dataingestion/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/k784648/miniconda3/envs/dataingestion/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

Scale the population percentages to [0,1]

In [233]:
numeric_columns = ['White', 'Black', 'Hispanic/Latino2', 'AIAN1', 'Asian', 'Mixed Race']

for column in numeric_columns:
    unique_crimes_df[column] = unique_crimes_df[column].map(lambda x: x/100)

/Users/k784648/miniconda3/envs/dataingestion/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [235]:
test = unique_crimes_df['UCRCrime']

In [245]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [246]:
le = LabelEncoder()

In [249]:
test_sparse = list(test)

In [255]:
test_num = le.fit_transform(test_sparse)

In [257]:
enc = OneHotEncoder()

In [261]:
one_hot = enc.fit_transform(test_num.reshape(-1,1))

In [266]:
one_hot

<336615x26 sparse matrix of type '<class 'numpy.float64'>'
	with 336615 stored elements in Compressed Sparse Row format>

array(['Aggravated Assault', 'Arson', 'Burglary', 'Criminal Homicide',
       'DUI', 'Disorderly Conduct', 'Drug Abuse Violations',
       'Drunkenness', 'Embezzlement', 'Forcible Rape',
       'Forgery and Counterfeiting', 'Fraud', 'Gambling', 'Larceny-theft',
       'Liquor Laws', 'Motor vehicle theft',
       'Offenses Against the Family and Children', 'Other',
       'Other assaults', 'Prostitution and Commercialized Vice',
       'Robbery', 'Sex Offenses', 'Stolen Property', 'Vagrancy',
       'Vandalism', 'Weapons'], dtype='<U40')